In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print('No Api Key was found')
elif not api_key.startswith('sk-proj-'):
    print("An api key was found, but it doesnt start with sk-proj")
elif api_key.strip() != api_key:
    print("An api key was found, but it might have space in the first or end")
else:
    print("Api key found and looks good so far!")

In [ ]:
openai = OpenAI()

In [ ]:
message = "Hello GPT, this is my first message"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content":message}])
print(response.choices[0].message.content)

In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
github = Website("https://github.com/Fikriraihan")
print(github.title)
print(github.text)

In [ ]:
system_prompt = "You are a skilled GitHub profile analyzer. " \
"Your job is to take the provided GitHub profile or repository URL and generate a clear, structured summary covering these points: " \
"1️⃣ **Profile Summary** " \
"- Username " \
"- Bio (if available) " \
"- Total public repositories " \
"- Total followers " \
"- Total stars received (sum across repos) " \
"- Top programming languages (by repo count) " \
"2️⃣ **Repository Highlights** (top 3 by stars or activity) " \
"For each: " \
"- Repository name " \
"- Description " \
"- Primary language " \
"- Star count " \
"- Last updated date " \
"- Notable technologies or frameworks used " \
"3️⃣ **Overall Assessment** " \
"- What does this user specialize in? " \
"- Are they more focused on personal projects or collaborations? " \
"- Any standout strengths or skills you notice? " \
"4️⃣ **Recommendations** " \
"- Suggest one area or technology they could explore next to grow. " \
"- Suggest one improvement to make their GitHub profile more appealing. " \
"Be concise, insightful, and encourage the user’s growth. " \
"If some data is missing, state it clearly instead of guessing."


In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a github named {website.title}"
    user_prompt += "\nThe contents of this github is as follows; \
please provide a summary of this website in markdown."
    user_prompt += website.text
    return user_prompt

In [ ]:
system_prompt

In [ ]:
print(user_prompt_for(github))

In [ ]:
messages_for(github)

In [ ]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://github.com/Fikriraihan")

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://github.com/Fikriraihan")